In [1]:
from transformers import BertTokenizer, BertConfig, BertForSequenceClassification, AdamW, PretrainedConfig
from transformers import get_linear_schedule_with_warmup
from pymorphy2 import MorphAnalyzer
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk
from tqdm import tqdm
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
import torch
import time

import requests
import json
from apscheduler.schedulers.background import BackgroundScheduler
import atexit

import logging
from telegram import InlineKeyboardButton, InlineKeyboardMarkup
import telegram
from collections import Counter

In [2]:

def get_tokens(seqs):
    
    # очистка
    for ind, seq in enumerate(seqs):
        seq = re.sub(r"http\S+", "", seq)
        seq = re.sub(r"http", "", seq)
        seq = re.sub(r"@\S+", "", seq)
        seq = re.sub(r"#(\w+)", "", seq)
        seq = re.sub(r"[^A-Za-zА-Яа-яё0-9\.\!\?\...]", " ", seq)
        seq = re.sub(r"[a-zа-я0-9]+\.[a-zа-я0-9]+\.*[a-zа-я0-9]*", "", seq)
        seq = re.sub(r"id\w+", "", seq)
        seq = re.sub(r"\@", "at", seq)
        seqs[ind] = seq
    
    # лемматизация и тоенизация
    X = []
    for seq in tqdm(seqs):
        new_seq = ["[CLS]"]
        for word in word_tokenize(seq):
            if word == '.':
                new_seq.append('[SEP]')
            elif '.' in word or len(re.findall(r'[0-9]+', word)) > 0:
                pass
            elif word not in russina_stop_words and word not in usa_stop_words:
                new_seq.append(pymorphy2_analyzer.parse(word)[0].normal_form)

        if new_seq[-1] != '[SEP]':
            new_seq.append('[SEP]')
        X.append(new_seq)

    # паддинги

    input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in X],
                maxlen=MAX_LEN, dtype="long", truncating="post", padding="post", value=3)

    # attention masks
    attention_masks = []
    for sent in input_ids:
        att_mask = [int(token_id != 3) for token_id in sent]
        # Store the attention mask for this sentence.
        attention_masks.append(att_mask)
        
        
    return torch.tensor(input_ids), torch.tensor(attention_masks)

In [21]:

nltk.download('stopwords')
from nltk.corpus import stopwords 
nltk.download('punkt')

pymorphy2_analyzer = MorphAnalyzer()
russina_stop_words = stopwords.words('russian')
usa_stop_words = stopwords.words('english')
tokenizer = BertTokenizer.from_pretrained('./vocab.txt')
MAX_LEN = 128
load_path = './bert_structEpoch1/'
bert = BertForSequenceClassification.from_pretrained(str(load_path))

[nltk_data] Downloading package stopwords to /home/feodor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/feodor/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


In [4]:
def predict(text):
    if not isinstance(text, list):
        text = [text]
    X, attention_masks = get_tokens(text)

    # пробуем
    bert.eval()
    preds = bert(X, 
        token_type_ids=None, 
        attention_mask=attention_masks)
    if not isinstance(text, list):
        return(np.argmax(preds[0].detach().numpy(), axis=1).flatten()[0])
    else:
        return(np.argmax(preds[0].detach().numpy(), axis=1).flatten())

In [5]:
def totg(texts):
    pred = predict(texts)
    return(list(pred))
#     for i, predres in enumerate(list(pred)):
#         prd = open('data/predictLog.csv','a')
#         prd.write("{};{};{}\n".format(predres,post[i]['text'][:250].replace('\n', '').replace('\r', '').replace(';','').replace('"',''),post[i]['wallUrl']))
#         prd.close
#         # json.dumps(post[i])
# #         postTg(post[i], predres)
#     print(predres)

In [6]:
file = open('data/predictLog.csv')

In [7]:
text = file.readlines()

In [22]:
ans = totg(text[:150])

100%|██████████| 150/150 [00:00<00:00, 163.01it/s]


In [23]:
Counter(ans)

Counter({0: 150})

In [ ]:
Counter